In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import torch
import torch.nn as nn
# from torchvision import models

In [4]:
%run func_DL.py

In [5]:
print(torch.cuda.is_available())

False


In [6]:
class model_task3(nn.Module):
    def __init__(self, in_channels, in_length, out_channels, out_length, batch_size):
        super().__init__()
        #-----------------------------------
        #var
        self.in_channels = in_channels
        self.in_length = in_length
        self.out_channels = out_channels
        self.out_length = out_length
        self.batch_size = batch_size
        #-----------------------------------
        #Conv1d
        #https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
        #https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md
        #conv1d_0
        kernel_size_conv1d_0=2; stride_conv1d_0=1; padding_conv1d_0=1; dilation_conv1d_0=1; 
        self.in_channels_conv1d_0 = self.in_channels; self.in_length_conv1d_0 = self.in_length
        self.out_channels_conv1d_0= 70; self.out_length_conv1d_0 = int((self.in_length_conv1d_0+2*padding_conv1d_0-dilation_conv1d_0*(kernel_size_conv1d_0-1)-1)/stride_conv1d_0+1)
        #conv1d_1
        kernel_size_conv1d_1=2; stride_conv1d_1=1; padding_conv1d_1=1; dilation_conv1d_1=1; 
        self.in_channels_conv1d_1 = self.out_channels_conv1d_0; self.in_length_conv1d_1 = self.out_length_conv1d_0
        self.out_channels_conv1d_1= 70; self.out_length_conv1d_1= int((self.in_length_conv1d_1+2*padding_conv1d_1-dilation_conv1d_1*(kernel_size_conv1d_1-1)-1)/stride_conv1d_1+1)
        #conv1d_2
        kernel_size_conv1d_2=2; stride_conv1d_2=1; padding_conv1d_2=1; dilation_conv1d_2=1; 
        self.in_channels_conv1d_2 = self.out_channels_conv1d_1; self.in_length_conv1d_2 = self.out_length_conv1d_1
        self.out_channels_conv1d_2= 70; self.out_length_conv1d_2= int((self.in_length_conv1d_2+2*padding_conv1d_2-dilation_conv1d_2*(kernel_size_conv1d_2-1)-1)/stride_conv1d_2+1)
        #conv1d_3
        kernel_size_conv1d_3=2; stride_conv1d_3=1; padding_conv1d_3=1; dilation_conv1d_3=1; 
        self.in_channels_conv1d_3 = self.out_channels_conv1d_2; self.in_length_conv1d_3 = self.out_length_conv1d_2
        self.out_channels_conv1d_3= 70; self.out_length_conv1d_3= int((self.in_length_conv1d_3+2*padding_conv1d_3-dilation_conv1d_3*(kernel_size_conv1d_3-1)-1)/stride_conv1d_3+1)
        #-----------------------------------
        #DNN
        self.in_channels_DNN = self.out_channels_conv1d_3*self.out_length_conv1d_3
        self.inner_channels_DNN = 70
        self.out_channels_DNN = self.out_channels*self.out_length

        #-----------------------------------
        #layer0
        self.layer0 = nn.Sequential(
            #-----------------------------------
            #conv1d_0
            nn.Conv1d(in_channels=self.in_channels_conv1d_0, out_channels=self.out_channels_conv1d_0, kernel_size=kernel_size_conv1d_0, stride=stride_conv1d_0, padding=padding_conv1d_0, dilation=dilation_conv1d_0),
            nn.BatchNorm1d(num_features=self.out_channels_conv1d_0),
            #nn.ReLU(),
            nn.Sigmoid(),
            #nn.LeakyReLU(0.2, inplace=True),
            #-----------------------------------
            #conv1d_1
            #nn.Dropout(p=0.2),
            nn.Conv1d(in_channels=self.in_channels_conv1d_1, out_channels=self.out_channels_conv1d_1, kernel_size=kernel_size_conv1d_1, stride=stride_conv1d_1, padding=padding_conv1d_1, dilation=dilation_conv1d_1),
            nn.BatchNorm1d(num_features=self.out_channels_conv1d_1),
            #nn.ReLU(),
            nn.Sigmoid(),
            #nn.LeakyReLU(0.2, inplace=True),
            #-----------------------------------
            #conv1d_2
            nn.Conv1d(in_channels=self.in_channels_conv1d_2, out_channels=self.out_channels_conv1d_2, kernel_size=kernel_size_conv1d_2, stride=stride_conv1d_2, padding=padding_conv1d_2, dilation=dilation_conv1d_2),
            nn.BatchNorm1d(num_features=self.out_channels_conv1d_2),
            #nn.ReLU(),
            nn.Sigmoid(),
            #nn.LeakyReLU(0.2, inplace=True),
            #-----------------------------------
            #conv1d_3
            nn.Conv1d(in_channels=self.in_channels_conv1d_3, out_channels=self.out_channels_conv1d_3, kernel_size=kernel_size_conv1d_3, stride=stride_conv1d_3, padding=padding_conv1d_3, dilation=dilation_conv1d_3),
            nn.BatchNorm1d(num_features=self.out_channels_conv1d_3),
            #nn.ReLU(),
            nn.Sigmoid(),
            #nn.LeakyReLU(0.2, inplace=True),
        )
        #-----------------------------------
        #layer1
        self.layer1 = nn.Sequential(
            #-----------------------------------
            nn.Linear(self.in_channels_DNN, self.inner_channels_DNN),
            nn.BatchNorm1d(num_features=self.inner_channels_DNN),
            #nn.ReLU(),
            #nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid(),
            #-----------------------------------
            #nn.Dropout(p=0.2),
            nn.Linear(self.inner_channels_DNN, self.inner_channels_DNN),
            #nn.ReLU(),
            #nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid(),
            #-----------------------------------
            nn.Linear(self.inner_channels_DNN, self.out_channels_DNN),
            #nn.ReLU(),
            nn.Sigmoid(),            
        )


    def forward(self, x):    
        output0 = self.layer0(x).view(-1, self.in_channels_DNN)
        output1 = self.layer1(output0).view(-1, self.out_channels, self.out_length)
        return output1

In [7]:
from torch.utils.data import Dataset, DataLoader

#---------------------------------------------
#var
path_dir_X = "../data_X"
path_dir_Y = "../data_Y_Task3"
n_test = 100
n_val = 100
batch_size = 100 #5000

#---------------------------------------------
#instance
dataset = CustomDataset(path_dir_X=path_dir_X, path_dir_Y=path_dir_Y, n_test=n_test, n_val=n_val, batch_size=batch_size)
dataloder = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [13]:
print(len(dataset))

15917


In [16]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print(train_size,val_size,val_size)

12733 3184 3184


In [8]:
#----------------------------
#var (condition)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#----------------------------
#var (train)
num_epochs = 1000
n_print_train_result = 1
val_flag = True

#----------------------------
#init (model)
in_channels = dataset.return_shape_X()[0]
in_length = dataset.return_shape_X()[1]
out_channels = dataset.return_shape_Y()[0]
out_length = dataset.return_shape_Y()[1]
model = model_task3(in_channels, in_length, out_channels, out_length, batch_size).to(device)
#init model weight
model.apply(init_normal_dist)
#----------------------------
#init (optimizer, scheduler)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,50,100,300,500], gamma=0.95)
#----------------------------
#init (loss_func)
#https://neptune.ai/blog/pytorch-loss-functions
loss_func = nn.MSELoss()
#loss_func = nn.L1Loss()

/g/g90/dhakal1/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [12]:
#----------------------------
#results
history = {"train_loss": [], "val_loss": []}

#----------------------------
#学習
for epoch in range(num_epochs):
  #----------------------------
  # train
  model.train()
  for i, (x, y) in enumerate(dataloder):
    #----------------------------
    #float32, grad==True
    x = dataset.change_data_setting_to_train(x)
    y = dataset.change_data_setting_to_train(y)
    print(x.shape,y.shape)
    #----------------------------
    #change the type
    x = x.to(device)
    y = y.to(device)
    #----------------------------
    #forward
    output = model(x)
    loss = loss_func(output, y)
    #----------------------------
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #----------------------------
    #print & result
    if (i+1) % n_print_train_result == 0:
      print(f'Epoch: {epoch+1}, iter: {i+1}, train_loss: {loss: 0.4f}')
      print(x.grad.mean())
  history["train_loss"].append(loss)

  #----------------------------
  # eval
  if val_flag == True:
    model.eval()
    with torch.no_grad():
      #----------------------------
      #forward
      x, y = dataset.return_val_data()
      #----------------------------
      #float32, grad==True
      x = dataset.change_data_setting_to_train(x)
      y = dataset.change_data_setting_to_train(y)
      #----------------------------
      #change the type
      x = x.to(device)
      y = y.to(device)
      #----------------------------
      #forward
      output = model(x)
      loss = loss_func(output, y)
      #----------------------------
      #print & result
      history["val_loss"].append(loss)
      print(f'Epoch: {epoch+1}, val_loss: {loss: 0.4f}')
  
  #----------------------------
  # scheduler
  scheduler.step()

torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
Epoch: 1, iter: 1, train_loss:  0.0078
tensor(-3.2734e-12)
torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
Epoch: 1, iter: 2, train_loss:  0.0079
tensor(-6.3276e-12)
torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
Epoch: 1, iter: 3, train_loss:  0.0068
tensor(1.2586e-12)
torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
Epoch: 1, iter: 4, train_loss:  0.0064
tensor(-2.7281e-12)
torch.Size([100, 12, 500]) torch.Size([100, 75, 1])


KeyboardInterrupt: 